In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import math

from esdglider import gcp, glider, utils

# deployment_info = {
#     "deployment": 'amlr08-20220513', 
#     "project": "SANDIEGO", 
#     "mode": 'delayed', 
#     "min_dt": '2022-05-13 18:56:55', 
# }
deployment_info = {
    "deployment": 'calanus-20241019', 
    "project": "ECOSWIM", 
    "mode": 'delayed', 
    "min_dt": '2024-10-19 17:37:00', 
}
# deployment_info = {
#     "deployment": 'amlr03-20231128', 
#     "project": "FREEBYRD", 
#     "mode": 'delayed', 
#     "min_dt": '2023-11-28 20:46', 
# }
deployment = deployment_info["deployment"]
project = deployment_info["project"]
mode = deployment_info["mode"]

year = utils.year_path(project, deployment)
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-config"

gcp.gcs_mount_bucket("amlr-gliders-deployments-dev", deployments_path, ro=False)


paths = glider.get_path_deployment(
    deployment_info, deployments_path, config_path)

dir_ts = paths["tsdir"]
path_sci = os.path.join(dir_ts, f"{deployment}-{mode}-sci.nc")
path_eng = os.path.join(dir_ts, f"{deployment}-{mode}-eng.nc")

# dir_ngdac = paths["profdir"]
# if os.path.isdir(dir_ngdac):
#     files_ngdac = os.listdir(dir_ngdac)
# files_ngdac
# os.path.join(paths["profdir"], f"{deployment} -{mode}.nc")
# path_sci = os.path.join(dir_ts, 'calanus-20241019-sci.nc')
# path_eng = os.path.join(dir_ts, 'calanus-20241019-eng.nc')


In [ ]:
ds_raw = xr.load_dataset(os.path.join(dir_ts, f"{deployment}-{mode}-raw.nc"))
df_raw = ds_raw.to_pandas()
display(ds_raw)

ds_eng = xr.load_dataset(path_eng)
df_eng = ds_eng.to_pandas()
display(ds_eng)

ds_sci = xr.load_dataset(path_sci)
df_sci = ds_sci.to_pandas()
display(ds_sci)

In [ ]:
import glidertools as gt
import pyglider.utils as pgutils
import math
vars_keep = ["longitude", "latitude", "depth", "profile_index", "profile_direction"]
x_orig = utils.data_var_reorder(ds_sci, vars_keep)
regions_df_orig = utils.calc_regions(x_orig)

# pgutils.get_profiles_new(x, filt_time=20)
p1, p2 = utils.findProfiles(
        x_orig.time.values,
        x_orig.depth.values,
        stall=0.5,
        shake=0,
        inversion = 10, 
        interrupt = 600,
        period = 60, 
        length = 8, 
    )

orig = ["profile_index_orig", "profile_direction_orig"]
x = x_orig.rename_vars({"profile_index": orig[0], "profile_direction": orig[1]})
x["profile_index"] = ('time', p1, {})
x["profile_direction"] = ('time', p2, {})
x = utils.data_var_reorder(x, (vars_keep + orig))

# x["dives"] = ('time', gt.utils.calc_dive_number(x.time.values, x.depth.values, dive_depth_threshold=15), {})
# display(x)

dt = "2024-11-09"
x2 = x.sel(time = slice(f"{dt} 12:00", dt))
x2_df = x2.to_pandas()
x2

In [ ]:
# print(np.argwhere(x.profile_direction.values == 0))

# print(len(x.time.values) - len(np.unique(x.time.values)))

x.profile_index.max()

In [ ]:
np.argwhere(regions_df_orig.profile_description == np.roll(regions_df_orig.profile_description, 1))

In [ ]:
regions_df = utils.calc_regions(x)
# display(regions_df)

# print(regions_df["profile_direction"].value_counts())
print("------------")
print(f"Climbs orig: {np.count_nonzero(regions_df_orig["profile_direction"] == -1)}")
print(f"Dives orig: {np.count_nonzero(regions_df_orig["profile_direction"] == 1)}")
utils.check_profiles(x_orig)

print("------------")
print(f"Climbs: {np.count_nonzero(regions_df["profile_direction"] == -1)}")
print(f"Dives: {np.count_nonzero(regions_df["profile_direction"] == 1)}")
utils.check_profiles(x)

regions_df
regions_df_orig

In [ ]:
prof_summ_df_orig = utils.check_profiles(x_orig)
prof_summ_df = utils.check_profiles(x)

between_df_orig = prof_summ_df_orig[prof_summ_df_orig["profile_index"] % 1 == 0.5]
between_df = prof_summ_df[prof_summ_df["profile_index"] % 1 == 0.5]

# print((between_df_orig.end_time - between_df_orig.start_time).describe())
# print((between_df.end_time - between_df.start_time).describe())

print(abs(between_df_orig.end_depth - between_df_orig.start_depth).describe())
print(abs(between_df.end_depth - between_df.start_depth).describe())

prof_summ_df

In [ ]:
# pgutils.get_profiles_new(x, filt_time=20)
e1, e2 = utils.findProfiles(
        ds_eng.time.values,
        ds_eng.depth.values,
        stall=20,
        shake=20,
        inversion = math.inf, 
        interrupt = math.inf,
    )

ds_eng["profile_index"] = ('time', e1, {})
ds_eng["profile_direction"] = ('time', e2, {})

regions_df_eng = utils.calc_regions(ds_eng)

print(f"Climbs: {np.count_nonzero(regions_df_eng["profile_direction"] == -1)}")
print(f"Dives: {np.count_nonzero(regions_df_eng["profile_direction"] == 1)}")
regions_df_eng

In [ ]:
# pgutils.get_profiles_new(x, filt_time=20)
r1, r2 = utils.findProfiles(
        ds_raw.time.values,
        ds_raw.depth.values,
        stall=20,
        shake=20,
        inversion = math.inf, 
        interrupt = math.inf,
    )

r1
pd.DataFrame(r1).describe()


ds_raw["profile_index"] = ('time', r1, {})
ds_raw["profile_direction"] = ('time', r2, {})

regions_df_raw = utils.calc_regions(ds_raw).assign(
    profile_direction_str=lambda d: d["profile_direction"].map(utils.direction_mapping), 
)
regions_df_raw

In [ ]:
diffs = ds_sci.time.to_pandas().diff().dt.total_seconds().round(0)
# diffs_in_seconds = np.array([d.total_seconds() for d in diffs])
# diffs_in_seconds
diffs.value_counts()